In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [1]:
def best_particle(X, objective_function):
    values = objective_function(X)
    best_particle = X[np.argmin(values)]
    return best_particle

In [3]:
def Reflection_spherical(X_old, V, radius,reflection_counter):
    global h
    N_particles = X_old.shape[0]
    X_new = X_old + h * V
    
    # Calculate norms for all particles
    norms = np.linalg.norm(X_new, axis=1)
    
    # Identify particles that need reflection
    reflect_mask = norms > radius
    original_counter = reflection_counter[0]
    reflection_counter[0] = original_counter + np.sum(reflect_mask)
    
    if np.any(reflect_mask):
        a = X_old[reflect_mask]
        b = V[reflect_mask]
        
        # Vectorized calculations
        p = 2 * np.sum(a * b, axis=1) / np.sum(b**2, axis=1)
        q = (np.sum(a**2, axis=1) - radius**2) / np.sum(b**2, axis=1)
        # Add epsilon and use np.maximum to avoid negative values under sqrt
        tau = -p/2 + np.sqrt(p**2/4 - q)
        NaN_mask = np.isnan(tau)
        num_nan = np.sum(NaN_mask)
        a[NaN_mask] = np.random.uniform(-512, 512, (num_nan, 2))
        b[NaN_mask] = np.random.uniform(-1, 1, (num_nan, 2))
        
        X_tau = a + tau[:, np.newaxis] * b
        
        # Calculate outward normals
        outward_normal = X_tau / np.linalg.norm(X_tau, axis=1)[:, np.newaxis]
         #the outward normal is calculated as the normalized position vector of the intersection point X_tau.
        #This works because for a circular (or spherical) boundary centered at the origin, 
        # the outward normal at any point on the boundary is simply the unit vector pointing from the origin to that point.
        
        # Calculate reflected velocities
        V_hat = b - 2 * np.sum(b * outward_normal, axis=1)[:, np.newaxis] * outward_normal
        
        # Update positions for reflected particles
        X_new[reflect_mask] = X_tau + (h - tau[:, np.newaxis]) * V_hat
        V[reflect_mask] = V_hat
    
    return X_new, V


In [2]:
def reflection_square(X_old, V, square_length, reflection_counter):
    global h
    X_new = X_old + h * V
    
    # Create masks for boundary crossings
    reflect_x = (np.abs(X_new[:, 0]) > square_length)
    reflect_y = (np.abs(X_new[:, 1]) > square_length)
    
    # Update reflection counter
    original_counter = reflection_counter[0]
    reflection_counter[0] = original_counter + np.sum(reflect_x) + np.sum(reflect_y)
    
    # Process X-axis reflections
    if np.any(reflect_x):
        a_x = X_old[reflect_x]
        b_x = V[reflect_x]
        
        # Calculate intersection times for X-boundary
        Vx = b_x[:, 0]
        boundary_x = np.where(Vx > 0, square_length, -square_length)
        tau_x = (boundary_x - a_x[:, 0]) / Vx
        
        # Handle Vx=0 cases (particles already outside boundary)
        tau_x = np.where(Vx == 0, h, tau_x)
        
        # Move to boundary and reflect velocity
        X_tau_x = a_x.copy()
        X_tau_x[:, 0] = boundary_x
        V_hat_x = b_x.copy()
        V_hat_x[:, 0] *= -1
        
        # Update positions and velocities
        X_new[reflect_x] = X_tau_x + (h - tau_x[:, None]) * V_hat_x
        V[reflect_x] = V_hat_x
    
    # Process Y-axis reflections
    if np.any(reflect_y):
        a_y = X_old[reflect_y]
        b_y = V[reflect_y]
        
        # Calculate intersection times for Y-boundary
        Vy = b_y[:, 1]
        boundary_y = np.where(Vy > 0, square_length, -square_length)
        tau_y = (boundary_y - a_y[:, 1]) / Vy
        
        # Handle Vy=0 cases
        tau_y = np.where(Vy == 0, h, tau_y)
        
        # Move to boundary and reflect velocity
        X_tau_y = a_y.copy()
        X_tau_y[:, 1] = boundary_y
        V_hat_y = b_y.copy()
        V_hat_y[:, 1] *= -1
        
        # Update positions and velocities
        X_new[reflect_y] = X_tau_y + (h - tau_y[:, None]) * V_hat_y
        V[reflect_y] = V_hat_y
    
    return X_new, V


In [ ]:
def Kalman_Langevin(X_old, V_old,objective_function, Lambda, gamma, sigma,radius, reflection_counter, square = True):
    # X: particle positions
    # weighted_average_X: weighted average of particle positions
    # V: particle velocities
    # Lambda: interaction coefficient
    # gamma: inertia coefficient
    # sigma: exploration coefficient
    # returns updated X, weighted_average_X, V
    N_particles = X_old.shape[0]

    global h
    if square == True:
        X_hat, V_hat = reflection_square(X_old,V_old,radius,reflection_counter)
    else:
        X_hat, V_hat = Reflection_spherical(X_old,V_old,radius,reflection_counter)
    
    X_new = X_hat
    best_particle_X = best_particle(X_hat,objective_function)
    #weighted_average_X = weighted_average(X_new,alpha,objective_function)
    V_new = V_hat - Lambda*(X_new-best_particle_X)*h-gamma*V_hat*h+sigma*np.sqrt(h)*np.random.randn(N_particles,1)
    
    return X_new, V_new